# **Analytical solution for 2D instantaneous solute transport injection in porous media (with decay)**


For this section, we'll see the evolution in time of the concentracion of five different contaminants, each with a different $\lambda$ value.

- 0.0001 day⁻¹ → very slow decay (e.g., nitrate in oxic groundwater, half-life ≈ 19 years)
- 0.001 day⁻¹ → moderate-slow decay (e.g., chlorinated solvents, half-life ≈ 1.9 years)
- 0.01 day⁻¹ → moderate decay (e.g., BTEX compounds biodegradation, half-life ≈ 70 days)
- 0.1 day⁻¹ → relatively fast decay (e.g., pathogens, half-life ≈ 1 week)
- 1.0 day⁻¹ → very fast decay (e.g., labile organic matter, half-life ≈ 0.7 days)

And the equation that defines 2D instantaneous solute transport injection, with decay, in porous media is the following,

$C(x, y, t) = \frac{C_{0}A}{4\pi t \sqrt{D'_{L}D'_{T}}}\operatorname{exp}\!\left( - \frac{\left( x-v't \right)^{2}}{4D'_{L}t} - \frac{y^{2}}{4D'_{T}t}\right)\operatorname{exp}\left( - \lambda t \right)$

**References:**
- Bear, J., & Cheng, A. H. D. (2010). Modeling groundwater flow and contaminant transport (Vol. 23, p. 834). Dordrecht: Springer.
- Bear, J. (2012). Hydraulics of groundwater. Courier Corporation.

<br>

## **1. Loading the needed libraries**

In [1]:
#-- Check and install required packages if not already installed 
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")
if_require("openpyxl")
if_require("xlrd")
if_require("numpy")
if_require("scipy")
if_require("os")

#-- Import the rest of libraries
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import scipy as sp
import os

from IPython.display import display, clear_output, HTML
from ipywidgets import Layout, Output
from tkinter import Tk, filedialog

import warnings
warnings.filterwarnings("ignore")

#-- LaTeX configuration, if installed
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "Computer Modern Roman"
# })

#-- Widgets parameters configuration
param_widgets = { 
                 'readout_format': ".2f", 
                 'style': {'description_width': '200px'}, 
                 'layout': widgets.Layout(width='500px')
                 }  

## **2. Coding the equations and plotting the results**

In [2]:

def effective_velocity(v: float, R:float) -> float:
    """Calculates effective velocity.

    Parameters
    ----------
    v : float
        velocity [L/T]
    R : float
        Retardation factor [-]

    Returns
    -------
    float
        Effective velocity [L/T]
    """
    
    v_prime = v / R
    return v_prime

def effective_dispersion(v: float, R: float, alpha: float, alpha_tran: float | None = None) -> float | tuple[float, float]:
    """Calculates effective dispersion coefficients considering retardation.
    
    When dispersion coefficient is None, then the function would give a value for both the longitudinal
    and transversal dispersion. When these two latter parameters get None, then dispersion coefficient 
    would be returned. 

    Parameters
    ----------
    v : float
        Groundwater velocity [L/T]
    R : float
        Retardation [-]
    alpha : float
        Dispersivity [L]
    alpha_tran : float | None, optional
        Transversal dispersivity, by default None (when 1D)

    Returns
    -------
    float | tuple[float, float]
        - If only `alpha` and `R` are provided:
            Effective longitudinal dispersion D' [L²/T].
        - If `alpha` and `alpha_tran` are provided:
            Tuple (Effective longitudinal dispersion, Effective transversal dispersion) where each 
            coefficient has been divided by R.
        
    Notes
    -----
    The retardation factor R must be ≥ 1. For R = 1, no retardation occurs and
    effective dispersion equals the original dispersion coefficients.
    """
    
    D_prime = (alpha*v) / R
    if alpha_tran is None:
        return D_prime
    else:
        D_tran_prime = (alpha_tran*v) / R
        return D_prime, D_tran_prime

#-- Instantaneous Injection: Solving solute transport in an infinite 2D domain with point source
def twoD_instantaneous_inj(Ci: float, A: float, x: float, y: float, t: np.ndarray, v: float, alpha: float, 
                           alpha_tran: float, R: float, decay: float) -> np.ndarray:
    """Calculates the concentration in a two-dimensional instantaneous injection framework.

    Parameters
    ----------
    Ci : float
        Initial concentration [M/V]
    A : float
        Contaminated area [L²]
    x : float 
        Position in x axis [L]
    y : float
        Position in y axis [L]
    t : np.ndarray
        Time
    v : float
        Groundwater velocity [L/T]
    alpha : float
        Longitudinal dispersivity [L]
    alpha_tran : float
        Transversal dispersivity [L]
    R : float
        Retardation [-]
    decay : float
        Coefficient of radioactive decay [1/T]

    Returns
    -------
    np.ndarray
        Concentration [M/V]
    """
    
    v_prime = effective_velocity(v, R)
    D_long_prime, D_tran_prime = effective_dispersion(v, R, alpha, alpha_tran)
    
    term1 = (Ci * A) / (4*np.pi*t*np.sqrt(D_long_prime*D_tran_prime))
    term2 = np.exp( - ( ((x-(v_prime*t))**2) / (4*D_long_prime*t) ) - ((y**2)/(4*D_tran_prime*t)) ) 
    term3 = np.exp(-decay*t)
    
    C = term1 * term2 * term3
    C_max = term1 * term3
    return {"C":C, "C_max": C_max}

In [ ]:
def find_parameters(x, y, alpha, alpha_tran, R):   
            
        decay_values = [0.0001, 0.001, 0.01, 0.1, 1]
        names_list = ["0.0001 [1/d]", "0.001 [1/d]", "0.01 [1/d]", "0.1 [1/d]", "1 [1/d]"]
               
        results = [twoD_instantaneous_inj(0.864, 500, x, y, np.arange(0, 365*2), 1, alpha, alpha_tran, R, decay) for decay in decay_values]
 
        #-- Plotting the results
        fig, ax1 = plt.subplots(figsize=(10, 3.5))
                        
        for contaminant in range(len(results)):
                ax1.plot(np.arange(0, 365*2), results[contaminant]["C"], label=names_list[contaminant])
        ax1.set_xlim(0, 365*2)
        # ax1.set_ylim(0, 1)
        ax1.set_xlabel(r'time [days]')
        ax1.set_ylabel('Concentration')
        ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        box = ax1.get_position()
        ax1.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        
sliders = [
        widgets.FloatSlider(value=100, min=10, max=1000, step=10, description=r'Distance x to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=0, min=0, max=500, step=10, description=r'Distance y to obs. point [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=0.2, max=10, step=0.1, description=r'Longitudinal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=0.1, min=0.02, max=1, step=0.001, description=r'Transversal dispersivity [m]', **param_widgets),
        widgets.FloatSlider(value=1, min=1, max=5, step=0.1, description=r'Retardation [-]', **param_widgets)
        ]

ui = widgets.HBox([widgets.VBox(sliders)])

out = widgets.interactive_output(find_parameters, {
        'x': sliders[0],
        'y': sliders[1],
        'alpha': sliders[2],
        'alpha_tran': sliders[3],
        'R': sliders[4]
        })

display(ui, out)

Output()